In [1]:
import torchaudio
import torch

In [3]:
# load file at data/vcc2016_training/SF1/100001.wav
waveform, sample_rate = torchaudio.load('../data/vcc2016_training/SF1/100001.wav')
waveform.size()

torch.Size([1, 56314])

# TODO:
- Just try to implement the architecture of CycleGAN
  - <<Here>>
- Then run a single training loop
- Once that works, train the network with SGD
  - This might require actually implementing the dataset

# Next:
- Finish downsample block

In [9]:
class Downsample(torch.nn.Module):
   
    def __init__(self, num_channels, kernel_size, stride):
      self.conv = torch.nn.Conv2d(in_channels=num_channels,
                                  out_channels=num_channels,
                                  kernel_size=kernel_size,
                                  stride=stride)
      self.norm = torch.nn.InstanceNorm2d(num_features=num_channels, affine=True)
      self.glu = torch.nn.GLU(dim=1)
    
    def forward(self, x):
      x = self.conv(x)
      x = self.norm(x)
      x = self.glu(x)
      return x
  
class ResidualBlock(torch.nn.Module):
  def __init__(self, num_channels1, num_channels2, kernel_size, stride):
    self.conv1 = torch.nn.Conv2d(in_channels=num_channels1,
                                out_channels=num_channels1,
                                kernel_size=kernel_size,
                                stride=stride)
    self.norm1 = torch.nn.InstanceNorm2d(num_features=num_channels1, affine=True)
    self.conv2 = torch.nn.Conv2d(in_channels=num_channels2,
                                out_channels=num_channels2,
                                kernel_size=kernel_size,
                                stride=stride)
    self.norm2 = torch.nn.InstanceNorm2d(num_features=num_channels2, affine=True)
  
  def forward(self, x):
    residual = x.clone()
    x = self.conv1(x)
    x = self.norm1(x)
    x = self.glu(x)
    x = self.conv2(x)
    x = self.norm2(x)
    return x + residual

class Upsample(torch.nn.Module):
    
    def __init__(self, num_channels, kernel_size, stride):
      self.conv = torch.nn.Conv2d(in_channels=num_channels,
                                          out_channels=num_channels,
                                          kernel_size=kernel_size,
                                          stride=stride)
      self.pixel_shuffle = torch.nn.PixelShuffle(upscale_factor=2)
      self.norm = torch.nn.InstanceNorm2d(num_features=num_channels, affine=True)
      self.glu = torch.nn.GLU(dim=1)
    
    def forward(self, x):
      x = self.conv(x)
      x = self.pixel_shuffle(x)
      x = self.norm(x)
      x = self.glu(x)
      return x

class Generator(torch.nn.Module):
    
    def __init__(self):
      self.conv1 = torch.nn.Conv2d(in_channels=128, out_channels=128, kernel_size=(1, 5), stride=(1, 2))
      self.glu = torch.nn.GLU(dim=1)
      self.downsample_twice = torch.nn.Sequential(
        Downsample(num_channels=256, kernel_size=(1, 5), stride=(1, 2)),
        Downsample(num_channels=512, kernel_size=(1, 5), stride=(1, 2))
      )
      self.residual_blocks = torch.nn.Sequential(
        [ResidualBlock(num_channels1=1024, num_channels2=512, kernel_size=(1, 3), stride=(1, 1)) for _ in range(6)]
      )
      self.upsample_twice = torch.nn.Sequential(
         Upsample(num_channels=1024, kernel_size=(1, 5), stride=(1, 1)),
         Upsample(num_channels=512, kernel_size=(1, 5), stride=(1, 1)),
      )
      self.conv2 = torch.nn.Conv2d(in_channels=24, out_channels=24, kernel_size=(1, 15), stride=(1, 1))

    def forward(self, x):
      x = self.conv1(x)
      x = self.glu(x)
      x = self.downsample_twice(x)
      x = self.residual_blocks(x)
      x = self.upsample_twice(x)
      x = self.conv2(x)
      return x

In [10]:
generator = Generator()

AttributeError: cannot assign module before Module.__init__() call